<h2 style="text-align: center; background-color: #4d648d; font-family:Arial; color: white; padding: 13px; line-height: 1; border-radius:10px"> 02_02Chargement et test de modèle </h2>

In [ ]:
# Étape 1 : Installation du package mistralai
!pip install mistralai

In [ ]:
# Étape 2 : Importer les bibliothèques nécessaires
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage
import os

# Execution via API

In [ ]:
# Étape 3 : Définir votre clé API 
os.environ["MISTRAL_API_KEY"] = "XXXXXXXXXXXXXXXXXXXXXXx"
api_key = os.environ["MISTRAL_API_KEY"]

In [ ]:
# Étape 4 : Initialiser le client Mistral
client = MistralClient(
    api_key=api_key,
)

models = client.list_models()
print("Modèles disponibles :", [m.id for m in models.data])

In [ ]:
# Étape 5 : test
messages = [
    ChatMessage(role="user", content="Explique-moi la théorie de la relativité en deux phrases.")
]

response = client.chat(
    model="mistral-tiny",  # ou mistral-small / mistral-medium 
    messages=messages
)

print("Réponse du modèle :", response.choices[0].message.content)

# Execution en local avec HuggingFace

In [ ]:
# Étape 1 : Installer les bibliothèques nécessaires
!pip install transformers accelerate torch

In [ ]:
# Étape 2 : Importer les modules
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

In [ ]:
# Étape 3 : Choisir et charger un modèle Mistral open source
# Ici on utilise "mistralai/Mistral-7B-Instruct-v0.2"
model_id = "mistralai/Mistral-7B-Instruct-v0.2"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,       # ou torch.float32 si pas de GPU
    device_map="auto"                # gère automatiquement l’allocation (CPU ou GPU)
)

In [ ]:
# Étape 4 : Préparer une requête utilisateur
prompt = "Explique-moi la théorie de la relativité en une phrase."

input_ids = tokenizer(prompt, return_tensors="pt").to(model.device)

In [ ]:
# Étape 5 : Générer une réponse avec le modèle
output = model.generate(
    **input_ids,
    max_new_tokens=100,
    temperature=0.7,
    top_p=0.9,
    do_sample=True
)

response = tokenizer.decode(output[0], skip_special_tokens=True)
print(response)
